<a href="https://www.kaggle.com/code/omarmostafataha/data-collection-project?scriptVersionId=143190226" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# For Arab football fans

This is a Python code to collect data about matches played in 2023 from the YallaKora website. YallaKora is a popular Arabic-language sports news and information website that covers various sports, including football, basketball, and more. It provides news, live scores, match results, statistics, and other sports-related content for audiences interested in sports, particularly in the Arab world and the Middle East. Users can find information about football leagues, teams, players, and matches, among other sports-related topics, on YallaKora website.

In [ ]:
def card_matches(champ_card, input_date):
    
    # extract card elements --> champ title and matches (title-->div, matches-->ul)
    card_contents = champ_card.contents
    card_contents = [item for item in card_contents if item != '\n']
    
    title_div = card_contents[0]
    matches_list = card_contents[1]
    
    # extract champ title from title_div
    title = title_div.find('h2').text.strip()
    
    # extract matches from matches_list 
    matches = matches_list.find_all('li')
    
    for match in matches:
        
        # extract each match data
        match_data = match.find('div',{'class':'teamsData'}).contents
        match_data = [item for item in match_data if item != '\n']
        teamA = match_data[1].text.strip() 
        teamB = match_data[5].text.strip()
        result_time = match_data[3].text.strip().replace('\n','',2).split('\n')
        
        # store match data in global lists
        team1.append(teamA)
        team2.append(teamB)
        result.append(result_time[0])
        time.append(result_time[1])
        championship.append(title)
        date.append(input_date)


In [ ]:
def scrap(input_date):
    # define the page you want to scrap
    url = f'https://www.yallakora.com/match-center/%D9%85%D8%B1%D9%83%D8%B2-%D8%A7%D9%84%D9%85%D8%A8%D8%A7%D8%B1%D9%8A%D8%A7%D8%AA?date={input_date}#'

    # sending request
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
    except Exception as e:
        print(f"An error occurred while making the request: {e}")
        
    if response.status_code == 200: 
        # extracting the page content
        content = response.content
        soup = BeautifulSoup(content,'lxml')

        # extract championship cards
        champ_cards = soup.find_all('div', {'class':'matchCard'})   

        # extract matches data
        for card in champ_cards:
            card_matches(card, input_date)
            
    else:
        print(f"Failed to retrieve the webpage [date: {input_date}] . Status code: {response.status_code}")


In [ ]:
# import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

# define empty lists 
team1 = []
team2 = []
result = []
time = []
championship = []
date = []

# Define a date range
start_date = datetime(2023, 1, 1)  # Start date (y,m,d)
end_date = datetime(2023, 12, 31)   # End date (y,m,d) 
current_date = start_date

# loop and scrap the pages in the date range
while current_date <= end_date:
    formatted_date = current_date.strftime("%m/%d/%Y")  # Format the date as needed
    scrap(formatted_date)  # Call the scrap function with the formatted date
    current_date += timedelta(days=1)  # Move to the next date

In [ ]:
# create a dataframe with all the lists
df = pd.DataFrame({'Championship':championship, 'Team 1':team1, 'Team 2':team2, 'Match Time':time, 'Match result':result, "Date":date})
df.head()

In [ ]:
df.shape

In [ ]:
# save the data in an excel file
df.to_excel('matches_2023.xlsx', index=False)